# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Задачи:**

* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

## Шаг 1. Изучение общей информации

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


**Вывод**
1. Данные в столбце days_employed имеют тип вещественного числа;
2. Значения стобца Образование имеют как строчные, так и прописные буквы;
3. Одни и те же цели прописаны несистемно (по-разному), что затрудняет их систематизацию. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**

Пропусков в столбцах days_employed и total_income одинковое количество. Скорее всего это связано с тем, что данные клиенты безработные, поэтому в данных столбцах рабочие дни и доход пропущены.  

In [4]:
#В столбце education приводим все значения к нижнему регистру
data['education'] = data['education'].str.lower()
print(data['education'])

0         высшее
1        среднее
2        среднее
3        среднее
4        среднее
          ...   
21520    среднее
21521    среднее
21522    среднее
21523    среднее
21524    среднее
Name: education, Length: 21525, dtype: object


In [5]:
#Заполним пропуски
data = data.fillna(0)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Теперь в датафрейме нет  пропусков, можно работать дальше

### Замена типа данных

In [6]:
#В столбце days_employed меняем тип данных с вещественного на целое число
data['days_employed'] = data['days_employed'].astype('int')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
None


**Вывод**

Для работы с данными в столбце days_employed удобнее их представить в виде целых чисел. Тем более что трудно воспринять день + сколько-то после запятой. 

### Обработка дубликатов

In [7]:
data.duplicated().sum()

71

In [8]:
data = data.drop_duplicates().reset_index(drop=True)

**Вывод**

Дубликатов обнаружено и удалено с заменой индекса - 71


Дубликаты могли возникнуть по причине того, что один и тот же человек мог взять несколько кредитов в течении определенного времени.

### Лемматизация

In [9]:
#Выводим уникальные значения столбца с целями кредита
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Одни и те же цели прописаны по-разному. Необходимо их лемматизировать и распределить по категориям. 

In [10]:
#Ищем леммы слов из столбца с целями кредита для того, чтобы распределить их по категориям
from pymystem3 import Mystem
m = Mystem()
#Делаем список purpose_list, чтобы внести туда уникальные значения столбца 'purpose' 
purpose_list = data['purpose'].unique()
#Слздаем переменную, в которую сохраним лемматизированные значения 
lemmas = []
#Каждую строчку в 'purpose_list' лемматизируем, получаем список лемм и добавляем значения в пустой список 'lemmas' и выводим на экран
for i in purpose_list:
    lemma = m.lemmatize(i)
    lemmas.append(lemma)
lemmas

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

**Вывод**

Предварительно цели кредита можно распределить по категориям:
1. Недвижимость (покупка, строительство);
2. Автомобиль;
3. Образование;
4. Свадьба

### Категоризация данных

In [11]:
#Теперь настало время распределить леммы по категорям. Создаем функцию, которая в зависимости от выделенной леммы определит
#категорию строки, а затем добавляем новый столбец в датафрейм data 
def purpose_change(purpose):
    lemmas_row = m.lemmatize(purpose)
    for i in lemmas_row:
        if 'авто' in i:
            return 'автомобиль'
        if 'недвиж' in i or 'жил' in i:
            return  'недвижимость'
        if 'свад' in i:
            return  'свадба'
        if 'образов' in i:
            return 'образование'

data['purpose_def']= data['purpose'].apply(purpose_change)        
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость


**Вывод**

С помощью лемматизации мы распределили по-разному сформулированные цели кредита по категорям и внесли их в отедльный столбец. Теперь можно работать с этими данными, чтобы ответить на вопрос о зависимости цели кредита и возврата этого кредита в срок.

## Шаг 3. Ответы на вопросы и решение задач

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Поскольку в датафрейме отмечено количество детей, а вопрос стоит о наличии или отсутствии таковых, то имеет смысл для удобства распределеить данные о детях на две категории: наличие и отсутствие детей.

In [12]:
#Напишем функцию, которая распределит данные по категориям наличие/отсутствие детей
# Значение 1 - наличие детей
# Значение 0 - отсутствие детей
def children_group(children):
    if children >= 1:
        return 1
    return 0
#Добавим новый столбец в датафрейм
data['children_group'] = data['children'].apply(children_group)
#Для нахождения зависимости воспользуемся инструментом pivot_table, он наглядно покажет как процентное соотношение, 
#так и конкретные цифры
pivot_table_children_group = data.pivot_table(index='children_group', columns= 'debt', values='children', aggfunc='count')
pivot_table_children_group['percent'] = pivot_table_children_group[1]/(pivot_table_children_group[1] \
                                                                       + pivot_table_children_group[0]) * 100
pivot_table_children_group

debt,0,1,percent
children_group,,,
0,13074,1064,7.525817
1,6639,677,9.253691


**Вывод**

Зависимость между наличием детей и возвратом кредита в срок наблюдается - люди с детьми имеют больший риск стать должниками банка.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
#Воспользуемся инструментом pivot_table
pivot_table_family_status = data.pivot_table(index='family_status', columns= 'debt', values='family_status_id', aggfunc='count')
pivot_table_family_status['percent'] = pivot_table_family_status[1]/(pivot_table_family_status[1] \
                                                                       + pivot_table_family_status[0]) * 100
pivot_table_family_status

debt,0,1,percent
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182


**Вывод**

Наблюдается зависимость между семейным положением и возвратом кредита в срок: 
Люди, состоящие в браке, имеют в среднем меньше задолженностей по возврату кредита в срок, нежели одинокие или состоящие в гражданском браке. А самый меньший процент задолженности по кредиту имеют вдовы/вдовцы.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос целесообразно распределить доходы из одноименного столбца по категориям: низкий, средний, высокий.
Возможный вариант выглядит следующим образом: 
1. Разделяем строки таблицы на 3, что составляет по 7 175 строк в каждой группе. Это будет равномерное распределение по группам.
2. Сохраняем данные о каждой группе в отдельной переменной.
3. В каждой переменной находим средние значения.
4. Создаем словарь и выводим значения по группам.
5. Делаем выводы.

In [14]:
sorted_income = data.sort_values(by='total_income', ascending=False)
sorted_income.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_def,children_group
12390,0,-1477,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью,недвижимость,0
19548,1,-2577,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости,недвижимость,1
9159,1,-5248,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование,образование,1
20742,0,-4719,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи,недвижимость,0
17137,0,-5734,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу,свадба,0
17460,0,-2285,43,среднее,1,женат / замужем,0,M,компаньон,0,1.597613e+06,операции с недвижимостью,недвижимость,0
18321,1,-333,41,высшее,0,гражданский брак,1,M,компаньон,0,1.551153e+06,свадьба,свадба,1
18306,1,-3173,41,высшее,0,Не женат / не замужем,4,F,компаньон,0,1.427934e+06,автомобиль,автомобиль,1
15234,1,-10207,64,высшее,0,в разводе,3,M,компаньон,0,1.350246e+06,жилье,недвижимость,1
11052,1,-1851,36,высшее,0,гражданский брак,1,F,сотрудник,0,1.286281e+06,покупка коммерческой недвижимости,недвижимость,1


In [15]:
high_income = sorted_income[['total_income','debt'][:7174]] 
high_income.mean()

total_income    151010.952271
debt                 0.081150
dtype: float64

In [16]:
medium_income = sorted_income.loc[7175:14350, ['total_income','debt']]
medium_income.mean()

total_income    110103.019180
debt                 0.084907
dtype: float64

In [17]:
low_income = sorted_income.loc[14351:, ['total_income','debt']]
low_income.mean()

total_income    64715.183359
debt                0.081081
dtype: float64

In [18]:
from json import dumps

income = {
    'высокий доход': high_income['debt'].mean() * 100,
    'средний доход': medium_income['debt'].mean() * 100,
    'низкий доход': low_income['debt'].mean() * 100
}
print('Процент долгов по кредиту в зависимости от уровня доходов:') 
print(dumps(income, indent=4, ensure_ascii=False))


Процент долгов по кредиту в зависимости от уровня доходов:
{
    "высокий доход": 8.115036822970076,
    "средний доход": 8.490660273698932,
    "низкий доход": 8.108108108108109
}


**Вывод**

Зависимости между уровнем дозода и возвратом кредита в срок не наблюдается. Значения по всем группам примерно равны, незначительные отклонения не влияют на общий фон.

### Как разные цели кредита влияют на его возврат в срок?

In [19]:
#Воспользуемся все тем же инструментом - сводными таблицами
pivot_table_purpose = data.pivot_table(index='purpose_def', columns= 'debt', values='purpose', aggfunc='count')
pivot_table_purpose['percent'] = pivot_table_purpose[1]/(pivot_table_purpose[1] \
                                                                       + pivot_table_purpose[0]) * 100
pivot_table_purpose

debt,0,1,percent
purpose_def,,,
автомобиль,3903,403,9.359034
недвижимость,10029,782,7.233373
образование,3643,370,9.220035
свадба,2138,186,8.003442


**Вывод**

Из полученных данных можно сделать следующий вывод: 
Те, кто берет кредит на недвижимость, более склонны вернуть его банку, а вот автомобиль и образование имеют значительный процент задолженности. 
А вот свадьба где-то посередине. Может быть потому, что кому-то все таки удается ее окупить подарками?

## Шаг 4. Общий вывод

1. Наблюдается зависимость между наличием\отсутствием детей и возвратом кредита в срок. Однако, исходя из полученных данных, можно сказать, что те, у кого есть дети, имеют все же больший процент долгов по кредиту, нежели те, у кого детей нет (9 против 7%). Скорее всего, это связано с тем, что те люди, у которых дети, в случае ухудшения финансового положения, в дилемме "заплатить по кредиту или накормить ребенка", выберут второе. Наличие детей является фактором дополнительных расходов, при котором эти самые расходы уменьшить очень тяжело даже при самом большом желании.


2. Наблюдается зависимость между семейным положением и возвратом кредита в срок. Тот факт, что люди, находящиеся в разводе, вдовцы/вдовы, а также семейные имеют меньший процент задолженности по кредитам, возможно объясняется тем фактом, что они либо имеют опыт сбалансированного семейного бюджета, либо имели его в прошлом. Люди без семьи, или живущие в гражданском браке, более склонны к спонтанным тратам, а также в целом к меньшему контролю за своим бюджетом.


3. Интересный вывод о том, что нет прицнипиальной зависимости между финансовым положением человека и возвратом кредита в срок можно объяснить тем фактом, что в каждой такой категории есть люди, у которых на возврат кредита влияют другие факторы: наличие/отсутствие детей, а также их семейное положение. Возможно они представленны в данных группах более менее равномерно.


4. Разные цели кредита все же влияют на возврат кредита в срок: те, кто берет кредит на недвижимость, все таки более склонны возвращать кредит в срок.

**Итог***

Резюмируя полученные данные и промежуточные выводы, можно сказать о том, что более склонными к возврату кредита в срок можно считать бездетных имеющих опыт семейной жизни, а также берущих кредит на недвижимость. 